In [ ]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import json
import time

In [ ]:
import numpy as np
import os
import sys

In [ ]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

### Twitter keys

In [ ]:
consumer_key="***"
consumer_secret="***"
access_token="***"
access_token_secret="***"

### Connect to database

In [ ]:
dbname = '***'
username = '***'

con = None
con = psycopg2.connect(database = dbname, user = username)

cur = con.cursor()

### Stream, process and store

In [ ]:
#Process hashtag component of incoming tweet
def hashtag_split(tweet_hashtags):
    tweet_hashtags_split = []
    for tag in tweet_hashtags:
        tweet_hashtags_split.append(str(tag['text']).lower())
    return ' '.join(tweet_hashtags_split)
        

In [ ]:
class StdOutListener(StreamListener):
    def __init__(self, time_limit):
        self.start_time = time.time()
        self.limit = time_limit
        super(StdOutListener, self).__init__()
    
    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            tweet = json.loads(data)
            try:
                #Takes timestamp, text and hashtags
                info = str(unicode(tweet['created_at']))+','+ str(unicode(tweet['text']))+','+ hashtag_split(tweet['entities']['hashtags'])
                cur.execute("INSERT INTO tweets VALUES ("+info+");")
                con.commit()
                
            except:
                pass
            return True
        else:
            return False

In [ ]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [ ]:
myStream = Stream(auth, listener=StdOutListener(time_limit='***'))
myStream.filter(track=['filter for election related keywords'])